# Preparation data

In [1]:
#choose which dfs you want to import
number='0'

In [2]:
# VODS disciplines codes:
discipline_codes=['0101',
'0102',
'0103',
'0104',
'0105',
'0106',
'0107',
'0199',
'0201',
'0202',
'0203',
'0204',
'0205',
'0206',
'0207',
'0208',
'0299',
'0301',
'0302',
'0303',
'0304',
'0305',
'0306',
'0399',
'0401',
'0402',
'0499',
'0501',
'0502',
'0503',
'0504',
'0505',
'0506',
'0507',
'0508',
'0599',
'0601',
'0602',
'0603',
'0604',
'0699',
'0700']

## Importing packages, data and functions

In [3]:
import pandas as pd
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import combinations 


In [4]:
# load distance matrix M (between disciplines)
path = r'C:\Users\lucp11051\Documents\Jupyter_notebooks\2_Discipline_distances/'


count_type = 'disc_in_a_person'
data = 'pub_2014'

M = np.loadtxt(path + 'cosine_distance_matrix_' + count_type + data+ '.txt')

In [5]:
path = r'C:\Users\lucp11051\Documents\Jupyter_notebooks\0_Preparing_dataframes/'

publications_0 = pd.read_csv(path + number+'publications_0.csv')  
projects_0 = pd.read_csv(path + number+'projects_0.csv')  
persons_0 = pd.read_csv(path + number+'persons_0.csv')  
organisations_0 = pd.read_csv(path + number+'organisations_0.csv')  

In [6]:
path = r'C:\Users\lucp11051\Documents\Jupyter_notebooks\0_Preparing_dataframes/'

publications = pd.read_csv(path + number+'publications.csv')  
projects = pd.read_csv(path + number+'projects.csv')  
persons = pd.read_csv(path + number+'persons.csv')  
organisations = pd.read_csv(path + number+'organisations.csv')  

In [7]:
# function to bring a list of (level 4) disciplines to a list of level 2
def bring_to_level_2(list_of_disc):
    level_2_list=[]
    for disc in list_of_disc:
        level_2_list= level_2_list + [disc[:4]]
    return level_2_list
        

In [8]:
# function to get unique values 
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    return unique_list

In [9]:
# function that adds columns to the initial dataframe df_0:
# 'Level-2 disciplines '+ entity_string: the level 2 disciplines of a chosen entity
# 'Unique level-2 disciplines '+ entity_string: the unique level 2 disciplines of a chosen entity
# Example: (df_0, entity_df, entity_string) = (persons,organisations, 'Organisations')
def get_related_level2_disciplines(df_0, entity_df, entity_string):
    level2_col=[]
    for i in range(len(df_0)):
        level2=[]
        for entity_id in df_0[entity_string][i]:
            if len(entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines']) == 0:
                entity_level2=[] 
            else:
                entity_level2=entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines'].iloc[0]
            level2=level2 + [entity_level2]
        level2_col=level2_col+[level2]
    df_0['Level-2 disciplines '+ entity_string]=level2_col
#     df_0['Unique level-2 disciplines '+ entity_string]=df_0['Level-2 disciplines '+ entity_string].apply(lambda x: unique(x))

In [10]:
# function takes a list with (non-unique) codes and gives a discipline vector (np.array)
def make_discipline_vector(level2_list, discipline_codes):
    disc_vector = np.array([0]*len(discipline_codes))
    for i in range(len(discipline_codes)):
        disc_vector[i]=level2_list.count(discipline_codes[i])
        
    if sum(disc_vector)>0:
        normalized_vector=disc_vector/sum(disc_vector)
    else:
        normalized_vector=disc_vector
    return normalized_vector
        
    

In [11]:
def difference_in_months(datetime_end, datetime_start):
    time_difference = relativedelta(datetime_end, datetime_start)
    difference_in_years = time_difference.years
    if time_difference.days > 14:
        extra_month = 1
    else:
        extra_month = 0
    rest_months = time_difference.months +extra_month
    difference_in_months = difference_in_years*12 + rest_months
    
    return difference_in_months

In [12]:
def string_to_datetime(date_str):
    if 'Z' in date_str:
        date_str=date_str.replace('Z','')
    return datetime.strptime(date_str, '%Y-%m-%d').date()

In [13]:
def merge_aliases(df, entity):
    new_df=df.copy()
    for i in range(df.shape[0]):        
        all_ids = [df['ID'][i]] + df['Alias'][i]
        alias_df = df[df['ID'].isin(all_ids)]
        if entity == 'projects':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Persons'] = str(unique(alias_df['Persons'].sum()))
            new_df.loc[i,'Organisations'] = str(alias_df['Organisations'].sum())
            new_df.loc[i,'Publications'] = str(unique(alias_df['Publications'].sum()))
            new_df.loc[i,'Duration'] = alias_df['Duration'].mean()
        elif entity == 'persons':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Organisations'] = str(unique(alias_df['Organisations'].sum()))
            new_df.loc[i,'Publications'] = str(unique(alias_df['Publications'].sum()))
            new_df.loc[i,'Projects'] = str(unique(alias_df['Projects'].sum()))
        elif entity == 'publications':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Persons'] = str(unique(alias_df['Persons'].sum()))               
            new_df.loc[i,'Size'] = alias_df['Size'].mean()
            new_df.loc[i,'Disciplines WoS'] = str(alias_df['Disciplines WoS'].sum())
            new_df.loc[i,'Disciplines vabb'] = str(alias_df['Disciplines vabb'].sum())
            new_df.loc[i,'WoS ID'] = str(alias_df['WoS ID'].sum())
            new_df.loc[i,'vabb ID'] = str(alias_df['vabb ID'].sum())
        else:
            raise Exception('Entity should be either \'projects\', \'persons\' or \'publications\'.')
    return new_df

In [14]:
def remove_bookcomponent(list_of_pubs):
    clean_list=[]
    for pub_id in list_of_pubs:
        pub_id = pub_id.replace('book-component:', '')
        clean_list=clean_list+[pub_id]
    return clean_list

## Preparing dataframes

In [15]:
# delete invalid and duplicate projects

projects_total=projects.shape[0]
projects=projects.drop(projects[projects.Name == 'ERROR'].index)
projects_valid=projects.shape[0]
projects.drop_duplicates(subset=['ID'])
projects.index=range(projects_valid)

print(projects_valid/projects_total *100, 'percent of projects is valid.')

projects_0_total=projects_0.shape[0]
projects_0=projects_0.drop(projects_0[projects_0.Name == 'ERROR'].index)
projects_0_valid=projects_0.shape[0]
projects_0.drop_duplicates(subset=['ID'])
projects_0.index=range(projects_0_valid)

print(projects_0_valid/projects_0_total *100, 'percent of projects_0 is valid.')

99.91645781119465 percent of projects is valid.
100.0 percent of projects_0 is valid.


In [16]:
# merge aliases, add level2 disciplines, literal_eval, ...

projects['Disciplines'] = projects['Disciplines'].apply(lambda x: literal_eval(x))
projects['Persons'] = projects['Persons'].apply(lambda x: unique(literal_eval(x)))
projects['Organisations'] = projects['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects['Publications'] = projects['Publications'].apply(lambda x: literal_eval(x))
projects['Alias'] = projects['Alias'].apply(lambda x: literal_eval(x))
projects['Start']=projects['Start'].apply(lambda x: string_to_datetime(x))
projects['End']=projects['End'].apply(lambda x: string_to_datetime(x))
length=[]
for i in range(projects.shape[0]):
    length=length + [difference_in_months(projects['End'][i],projects['Start'][i])]
projects['Duration']=length
projects = merge_aliases(projects, 'projects')

projects['Disciplines'] = projects['Disciplines'].apply(lambda x: literal_eval(x))
projects['Persons'] = projects['Persons'].apply(lambda x: unique(literal_eval(x)))
projects['Organisations'] = projects['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects['Publications'] = projects['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
projects['Level-2 disciplines']=projects['Disciplines'].apply(lambda x: bring_to_level_2(x))
projects['Size']=projects['Persons'].apply(lambda x: len(x))

In [17]:
# merge aliases, add level2 disciplines, literal_eval, ...
projects_0['Disciplines'] = projects_0['Disciplines'].apply(lambda x: literal_eval(x))
projects_0['Persons'] = projects_0['Persons'].apply(lambda x: unique(literal_eval(x)))
projects_0['Organisations'] = projects_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects_0['Publications'] = projects_0['Publications'].apply(lambda x: literal_eval(x))
projects_0['Alias'] = projects_0['Alias'].apply(lambda x: literal_eval(x))
projects_0['Start']=projects_0['Start'].apply(lambda x: string_to_datetime(x))
projects_0['End']=projects_0['End'].apply(lambda x: string_to_datetime(x))
length=[]
for i in range(projects_0.shape[0]):
    length=length + [difference_in_months(projects_0['End'][i],projects_0['Start'][i])]
projects_0['Duration']=length
projects_0 = merge_aliases(projects_0, 'projects')

projects_0['Disciplines'] = projects_0['Disciplines'].apply(lambda x: literal_eval(x))
projects_0['Persons'] = projects_0['Persons'].apply(lambda x: unique(literal_eval(x)))
projects_0['Organisations'] = projects_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects_0['Publications'] = projects_0['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
projects_0['Level-2 disciplines']=projects_0['Disciplines'].apply(lambda x: bring_to_level_2(x))
projects_0['Size']=projects_0['Persons'].apply(lambda x: len(x))

In [18]:
# delete invalid and duplicate persons

persons_total=persons.shape[0]
persons=persons.drop(persons[persons.Name == 'ERROR'].index)
persons_valid=persons.shape[0]
persons.drop_duplicates(subset=['ID'])
persons.index=range(persons_valid)

print(persons_valid/persons_total *100, 'percent of persons is valid.')

persons_0_total=persons_0.shape[0]
persons_0=persons_0.drop(persons_0[persons_0.Name == 'ERROR'].index)
persons_0_valid=persons_0.shape[0]
persons_0.drop_duplicates(subset=['ID'])
persons_0.index=range(persons_0_valid)

print(persons_0_valid/persons_0_total *100, 'percent of persons_0 is valid.')

100.0 percent of persons is valid.
100.0 percent of persons_0 is valid.


In [19]:
# merge aliases, add level2 disciplines, literal_eval, ...
persons['Disciplines'] = persons['Disciplines'].apply(lambda x: literal_eval(x))
persons['Projects'] = persons['Projects'].apply(lambda x: literal_eval(x))
persons['Organisations'] = persons['Organisations'].apply(lambda x: literal_eval(x))
persons['Publications'] = persons['Publications'].apply(lambda x: literal_eval(x))
persons['Alias'] = persons['Alias'].apply(lambda x: literal_eval(x))
persons = merge_aliases(persons, 'persons')
persons['Disciplines'] = persons['Disciplines'].apply(lambda x: literal_eval(x))
persons['Projects'] = persons['Projects'].apply(lambda x: unique(literal_eval(x)))
persons['Organisations'] = persons['Organisations'].apply(lambda x: unique(literal_eval(x)))
persons['Publications'] = persons['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
persons['Level-2 disciplines']=persons['Disciplines'].apply(lambda x: bring_to_level_2(x))


In [20]:
# merge aliases, add level2 disciplines, literal_eval, ...
persons_0['Disciplines'] = persons_0['Disciplines'].apply(lambda x: literal_eval(x))
persons_0['Projects'] = persons_0['Projects'].apply(lambda x: literal_eval(x))
persons_0['Organisations'] = persons_0['Organisations'].apply(lambda x: literal_eval(x))
persons_0['Publications'] = persons_0['Publications'].apply(lambda x: literal_eval(x))
persons_0['Alias'] = persons_0['Alias'].apply(lambda x: literal_eval(x))
persons_0 = merge_aliases(persons_0, 'persons')
persons_0['Disciplines'] = persons_0['Disciplines'].apply(lambda x: literal_eval(x))
persons_0['Projects'] = persons_0['Projects'].apply(lambda x: unique(literal_eval(x)))
persons_0['Organisations'] = persons_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
persons_0['Publications'] = persons_0['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
persons_0['Level-2 disciplines']=persons_0['Disciplines'].apply(lambda x: bring_to_level_2(x))


In [21]:
# delete invalid and duplicate publications

publications_total=publications.shape[0]
publications=publications.drop(publications[publications.Name == 'ERROR'].index)
publications_valid=publications.shape[0]
publications.drop_duplicates(subset=['ID'])
publications.index=range(publications_valid)

print(publications_valid/publications_total *100, 'percent of publications is valid.')

78.33859095688749 percent of publications is valid.


In [22]:
# merge aliases, add level2 disciplines, literal_eval, ...
publications['Size'] = publications['Size'].apply(lambda x: max(1,literal_eval(x)))
publications['WoS ID'] = publications['WoS ID'].apply(lambda x: literal_eval(x))
publications['vabb ID'] = publications['vabb ID'].apply(lambda x: literal_eval(x))
publications['Disciplines WoS'] = publications['Disciplines WoS'].apply(lambda x: literal_eval(x))
publications['Disciplines vabb'] = publications['Disciplines vabb'].apply(lambda x: literal_eval(x))
publications['Disciplines'] = publications['Disciplines'].apply(lambda x: literal_eval(x))
publications['Persons'] = publications['Persons'].apply(lambda x: unique(literal_eval(x)))
publications['Alias'] = publications['Alias'].apply(lambda x: literal_eval(x))
publications=merge_aliases(publications, 'publications')
publications['WoS ID'] = publications['WoS ID'].apply(lambda x: literal_eval(x))
publications['vabb ID'] = publications['vabb ID'].apply(lambda x: literal_eval(x))
publications['Disciplines WoS'] = publications['Disciplines WoS'].apply(lambda x: literal_eval(x))
publications['Disciplines vabb'] = publications['Disciplines vabb'].apply(lambda x: literal_eval(x))
publications['Disciplines'] = publications['Disciplines'].apply(lambda x: literal_eval(x))
publications['Persons'] = publications['Persons'].apply(lambda x: unique(literal_eval(x)))
publications['Level-2 disciplines']=publications['Disciplines'].apply(lambda x: bring_to_level_2(x))

In [23]:
# delete invalid and duplicate organisations

organisations_total=organisations.shape[0]
organisations=organisations.drop(organisations[organisations.Name == 'ERROR'].index)
organisations_valid=organisations.shape[0]
organisations.drop_duplicates(subset=['ID'])
organisations.index=range(organisations_valid)

print(organisations_valid/organisations_total *100, 'percent of organisations is valid.')

100.0 percent of organisations is valid.


In [24]:
organisations['Disciplines'] = organisations['Disciplines'].apply(lambda x: literal_eval(x))
organisations['Level-2 disciplines']=organisations['Disciplines'].apply(lambda x: bring_to_level_2(x))

## Projects

In [25]:
projects

,ID,Name,Start,End,Disciplines,Persons,Organisations,Publications,Alias,when_added,Duration,Level-2 disciplines,Size
0,fdde0090-f6ed-4938-91a6-13d370125cde,ICP Interuniversity Programme Physical land re...,2015-09-22,2016-09-30,"[04010303, 01070405, 01050313]",[b73e0e47-ad11-4ce7-a065-2993e6c3b602],"[2b317cdb-3a20-47d7-b2bd-75aa36eb9734, f28fd86...",[],[],0,12,"[0401, 0107, 0105]",1
1,0f80da13-5ac3-45c0-a0df-c2bd93051dbe,ERC Professorship GlycoTarget,2014-03-01,2019-02-28,"[010608, 030130, 030116, 030617, 030630, 03012...",[8834112c-33f1-4ee6-8a34-9c09a7b71535],[adbfa1d6-6a4b-4abf-8dc4-ea71c647af26],[],[],0,60,"[0106, 0301, 0301, 0306, 0306, 0301, 0106, 030...",1
2,0e185133-3ddf-4f8a-81d6-2ddc76a49cb6,EU Promotion of Democratic Governance via Func...,2016-07-01,2017-01-31,"[050604, 050602, 050699, 050607, 050606, 05060...","[f19619e2-340d-47ef-9792-35d2436febc4, e011bd3...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],[],[],0,7,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",2
3,fc7f8873-326f-41fe-9c75-08ea05d8058b,Somite development in 3D in vitro synthetic mi...,2018-01-01,2018-12-31,[010699],[05e00835-520a-4d3b-8e53-fbe03c1e43c7],[b9b3c61e-650e-4463-a0e2-6de8f49917ba],[],[],0,12,[0106],1
4,9be600ee-e306-4fac-8c4b-0e5134b9ff28,Housing for Refugee Inclusion exploring inclus...,2020-01-01,2023-12-31,"[02011199, 05040413, 05040702, 05060199, 06040...",[3de615bf-4167-4fa5-a4d3-0dac8248798a],[3420d399-29f4-4edf-9f9d-252e7b1d400d],[],[],0,48,"[0201, 0504, 0504, 0506, 0604]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,5177ba5f-64c4-4e9e-bf4e-d4eb5e4f25b9,Synthesis of benzodiocane ligin for applicatio...,2016-10-01,2019-09-30,"[020501, 020799, 01061007, 010606, 020601, 010...",[6c4f337a-6fac-41fc-9224-dc0ccbdeeebb],[37e63935-be43-4a88-8f83-97da12647a08],[],[],1,36,"[0205, 0207, 0106, 0106, 0206, 0106, 0302, 010...",1
1192,c641efba-1526-4018-80ba-2de9152d0e94,The soil systems under global change SOGLO,2012-10-01,2017-12-31,"[040103, 010704, 010503, 010604, 010701, 01070...","[bdf09bca-071b-4682-be43-6c47560ecab3, 47582bc...","[17cf03fe-441e-444b-87f4-62214f211431, f414cd4...",[],[fce55ffb-6026-455d-93c0-9389e0f52164],1,63,"[0401, 0107, 0105, 0106, 0107, 0107, 0107]",2
1193,6a2353c9-539f-47da-bc58-65f961a94d47,Congo basin integrated monitoring for forest c...,2010-12-15,2016-12-31,"[010601, 030505, 010401, 010606, 010610, 01060...","[9d445952-3146-4bce-8477-b3b681cf0b1f, bdf09bc...","[baf2328c-ac14-4990-8086-1126ac6cea11, 17cf03f...",[],[3ce1811f-5c7a-4854-8d8a-67e018819947],1,70,"[0106, 0305, 0104, 0106, 0106, 0106, 0107, 010...",3
1194,165ae5f2-aac4-4893-8597-240e351c60c7,ARBOREF integrated 39 lignin first 39 biorefinery,2015-04-01,2019-03-31,"[020602, 040102, 059999, 020404, 01061106, 020...","[a190c1ac-b82c-477b-8b16-39a5752d3e79, 6c4f337...","[b58d2d09-7716-4627-851a-d1608fec639f, 37e6393...",[],[c0c71b56-d67e-4445-870c-b6b7e9b0da77],1,48,"[0206, 0401, 0599, 0204, 0106, 0206, 0306, 020...",4


## projects_0

In [26]:
get_related_level2_disciplines(projects_0, persons, 'Persons')

get_related_level2_disciplines(projects_0, organisations, 'Organisations')

In [27]:
projects_0

,ID,Name,Start,End,Disciplines,Persons,Organisations,Publications,Alias,when_added,Duration,Level-2 disciplines,Size,Level-2 disciplines Persons,Level-2 disciplines Organisations
0,fdde0090-f6ed-4938-91a6-13d370125cde,ICP Interuniversity Programme Physical land re...,2015-09-22,2016-09-30,"[04010303, 01070405, 01050313]",[b73e0e47-ad11-4ce7-a065-2993e6c3b602],"[2b317cdb-3a20-47d7-b2bd-75aa36eb9734, f28fd86...",[],[],0,12,"[0401, 0107, 0105]",1,[[]],"[[0203, 0105, 0201], [0107, 0299, 0204, 0601, ..."
1,0f80da13-5ac3-45c0-a0df-c2bd93051dbe,ERC Professorship GlycoTarget,2014-03-01,2019-02-28,"[010608, 030130, 030116, 030617, 030630, 03012...",[8834112c-33f1-4ee6-8a34-9c09a7b71535],[adbfa1d6-6a4b-4abf-8dc4-ea71c647af26],[],[],0,60,"[0106, 0301, 0301, 0306, 0306, 0301, 0106, 030...",1,"[[0306, 0106, 0306, 0306, 0301, 0302, 0301, 03...","[[0106, 0106]]"
2,0e185133-3ddf-4f8a-81d6-2ddc76a49cb6,EU Promotion of Democratic Governance via Func...,2016-07-01,2017-01-31,"[050604, 050602, 050699, 050607, 050606, 05060...","[f19619e2-340d-47ef-9792-35d2436febc4, e011bd3...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],[],[],0,7,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",2,"[[0506, 0506, 0506], [0506, 0506, 0506, 0506, ...",[[0504]]
3,fc7f8873-326f-41fe-9c75-08ea05d8058b,Somite development in 3D in vitro synthetic mi...,2018-01-01,2018-12-31,[010699],[05e00835-520a-4d3b-8e53-fbe03c1e43c7],[b9b3c61e-650e-4463-a0e2-6de8f49917ba],[],[],0,12,[0106],1,[[0306]],"[[0301, 0306, 0301, 0306, 0301, 0304, 0306, 02..."
4,9be600ee-e306-4fac-8c4b-0e5134b9ff28,Housing for Refugee Inclusion exploring inclus...,2020-01-01,2023-12-31,"[02011199, 05040413, 05040702, 05060199, 06040...",[3de615bf-4167-4fa5-a4d3-0dac8248798a],[3420d399-29f4-4edf-9f9d-252e7b1d400d],[],[],0,48,"[0201, 0504, 0504, 0506, 0604]",1,"[[0201, 0201, 0604, 0604, 0604]]","[[0201, 0201]]"
5,ee843e8f-9c18-4c14-bfee-641e18fbd8e0,Lost tributaries of the Scheldt reconstructing...,2019-01-01,2019-12-31,"[06010119, 06010124, 06010108, 06010116]","[45230f30-5ba1-4ba1-a305-1307392c1cc1, acb40a5...",[eda251f4-4796-4ac5-a6de-0e58a70f58aa],[],[],0,12,"[0601, 0601, 0601, 0601]",2,"[[0601, 0105, 0105, 0601, 0601, 0601, 0601, 06...",[[0601]]
6,bf2e0a13-e0bf-48cd-8339-c8c256770b1c,The association between motor skill learning a...,2020-08-04,2024-08-04,"[03011804, 03011899, 03012306, 03012301, 03012...","[d86af692-304d-42dd-bb1f-86d0113be027, 24e9ba0...","[d4a49766-ed89-4de9-a06b-3c45f728354f, 3597372...",[],[],0,48,"[0301, 0301, 0301, 0301, 0301]",4,"[[0304], [0301, 0304, 0304, 0306], [0301, 0301...","[[0503], [0301, 0304, 0304, 0306]]"
7,7a8539e2-93ad-4864-a4c7-405ec848fdd4,Towards dynamic navigational charts for highly...,2020-09-04,2024-09-04,[02020301],"[e739bf7a-76c2-4e6e-8b6e-4dcf863be95f, de9b527...","[4ad02e7f-d5fa-4d66-98bb-d2f63929649a, 4c3d49c...",[],[],0,48,[0202],2,"[[0203], [0203]]","[[0203], [0203, 0203]]"
8,70295923-ab2c-44c6-ac9f-2ff4f28b5a75,Analysing the impact of head end tail end effe...,2020-05-14,2024-05-14,"[01070310, 04010302, 01050607]","[8c662e7b-3e22-4f37-bc2f-a4c7f8593181, 8d75503...","[f74d3c9c-2f54-4b95-9b41-61afcfe2a878, f1d3171...",[],[],0,48,"[0107, 0401, 0105]",3,"[[0105, 0106, 0107, 0107, 0201, 0202, 0299, 04...","[[0107], [0107]]"
9,c04fd85a-d90f-438d-b06a-e5606588ba84,2D materials optomechanics in Silicon photonics,2021-01-01,2021-06-30,"[01030999, 01030904]","[2e1cb018-8750-4c8e-a23b-a5244883a5a6, c405840...",[16a28d37-6005-49e4-be1b-46b8c76f6640],[],[],0,6,"[0103, 0103]",2,"[[0208, 0208, 0208, 0208], [0103, 0202, 0103, ...","[[0208, 0202, 0202]]"


## Persons

In [28]:
persons

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050..."
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030..."
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[]
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]"
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060..."
...,...,...,...,...,...,...,...,...,...
3769,fceb2b53-06c6-48a7-bad8-207e4c7df6d5,Herman Peiffer,"[020199, 020110]",[],[4bdf446c-b1de-4cb4-9872-07070924a4ee],"[1c2f77c9-1d61-4114-8e08-53cc526d0ecb, 45760ef...",[],1,"[0201, 0201]"
3770,a3855699-cf44-4e45-8ed6-d4c97b84c736,Jeroen De Maeyer,[02030104],[8cc324ad-6d0e-4f72-971b-9d0f4c7f454a],[5f3f66cc-4003-4831-9855-1a9db80e799b],[],[],1,[0203]
3771,1384722e-a9f8-469a-98f4-28b94091cd09,Ioan Mugurel Feraru,"[030303, 030602, 030301, 030202, 030102, 02080...",[],[37e63935-be43-4a88-8f83-97da12647a08],"[ce296dde-9ab7-4840-aa70-e3c57765f4ca, ac1f708...",[],1,"[0303, 0306, 0303, 0302, 0301, 0208, 0207, 020..."
3772,443c08c9-5928-493f-99d4-23d72b9ad0cf,Jo De Vrieze,"[01069999, 01060803, 02070299, 01060899, 02040...","[3ba09a79-3e6d-4dd9-9c84-bc37d9cc4dc1, 55dfba8...","[e4b3631f-abdd-45bf-a5e9-86c6f2e6e4d9, ce1ce9f...","[0a367678-55cf-4c06-bc75-96a5c83ff3c4, ba0ca1f...",[f1da3d6e-51f7-4a7a-b767-400bf91e016a],1,"[0106, 0106, 0207, 0106, 0204, 0204, 0207, 010..."


## Persons_0

In [29]:
# add total number of publication coauthors
persons_0['Coauthors Publications total count']=np.zeros(len(persons_0))
for i in range(len(persons_0)):
    coauthor_count=0
    for pub_id in persons_0['Publications'][i]:
        if len(publications[publications['ID'] == pub_id]['Size']) == 0:
            continue
        else:
            coauthor_count=coauthor_count+publications[publications['ID'] == pub_id]['Size'].iloc[0]-1
    persons_0.loc[i,'Coauthors Publications total count']=coauthor_count

In [30]:
persons_0

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines,Coauthors Publications total count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",6.0
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030...",816.5
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[],244.0
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]",376.0
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060...",117.0
...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,Wout Boerjan,[01061004],"[6c31ab62-2681-4e05-953a-2e439d440a8b, 1e306f8...",[37e63935-be43-4a88-8f83-97da12647a08],"[086fba17-4d80-4006-9f9d-52ed4d090e38, 860b4f9...",[],0,[0106],1144.0
60,4b74615b-d701-4448-a191-cb19436ee4cb,Dirk Vandepitte,[029999],"[017c8b7d-4efe-4647-97af-297bfd23595f, 4a7fc94...","[5f3f66cc-4003-4831-9855-1a9db80e799b, 22358ad...","[f9e8018c-49b3-4d66-9c12-4de3243c097b, e702163...",[],0,[0299],1186.0
61,b6fd60a4-c095-43ed-9743-67cd091db492,Lotte JANSSENS,"[03020301, 03022601, 03022602, 03040201, 03040...","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[172ee4c2-42bc-4e59-93f2-91f28a162f2a, eb6b8bd...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[64b0cfc6-e0a3-4efd-8576-480f849dec0d],0,"[0302, 0302, 0302, 0304, 0304, 0304, 0304, 030...",359.5
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,Nina Goossens,"[03040599, 030126, 030402, 030405, 030626]","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[07fce545-f735-4a6b-9b40-02814a257bca, b4b0de5...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[89a6c8c9-fe61-434e-b2be-402f07e8c751],0,"[0304, 0301, 0304, 0304, 0306]",157.5


In [31]:
persons_0

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines,Coauthors Publications total count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",6.0
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030...",816.5
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[],244.0
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]",376.0
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060...",117.0
...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,Wout Boerjan,[01061004],"[6c31ab62-2681-4e05-953a-2e439d440a8b, 1e306f8...",[37e63935-be43-4a88-8f83-97da12647a08],"[086fba17-4d80-4006-9f9d-52ed4d090e38, 860b4f9...",[],0,[0106],1144.0
60,4b74615b-d701-4448-a191-cb19436ee4cb,Dirk Vandepitte,[029999],"[017c8b7d-4efe-4647-97af-297bfd23595f, 4a7fc94...","[5f3f66cc-4003-4831-9855-1a9db80e799b, 22358ad...","[f9e8018c-49b3-4d66-9c12-4de3243c097b, e702163...",[],0,[0299],1186.0
61,b6fd60a4-c095-43ed-9743-67cd091db492,Lotte JANSSENS,"[03020301, 03022601, 03022602, 03040201, 03040...","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[172ee4c2-42bc-4e59-93f2-91f28a162f2a, eb6b8bd...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[64b0cfc6-e0a3-4efd-8576-480f849dec0d],0,"[0302, 0302, 0302, 0304, 0304, 0304, 0304, 030...",359.5
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,Nina Goossens,"[03040599, 030126, 030402, 030405, 030626]","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[07fce545-f735-4a6b-9b40-02814a257bca, b4b0de5...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[89a6c8c9-fe61-434e-b2be-402f07e8c751],0,"[0304, 0301, 0304, 0304, 0306]",157.5


In [32]:
organisations

,ID,Name,Disciplines,when_added,Level-2 disciplines
0,45ff2fc2-f78b-489c-ae38-b6fca2022521,Department of Political Sciences,[050404],0,[0504]
1,f28fd86e-fce7-4c98-90fb-015b1a4e5523,Earth System Sciences,"[010703, 029901, 020406, 060101, 060104, 06040...",0,"[0107, 0299, 0204, 0601, 0601, 0604, 0601, 020..."
2,2b317cdb-3a20-47d7-b2bd-75aa36eb9734,Hydrology and Hydraulic Engineering,"[020307, 010503, 020112]",0,"[0203, 0105, 0201]"
3,adbfa1d6-6a4b-4abf-8dc4-ea71c647af26,Department of Biochemistry and microbiology,"[010602, 010608]",0,"[0106, 0106]"
4,3420d399-29f4-4edf-9f9d-252e7b1d400d,"Urban Design, Urbanism, Landscape and Planning","[020109, 020111]",0,"[0201, 0201]"
...,...,...,...,...,...
85,9789e8e7-746a-46b5-8e3e-f8c5b6148e54,Mecha(tro)nic System Dynamics,[029999],1,[0299]
86,fd34a10e-71f6-4db9-acc0-238983fdb834,"Research Centre Accountancy, Leuven",[050201],1,[0502]
87,09df8bc5-bede-4232-8f5c-ece62f241fdc,"Leuven Centre for Aero & Space Science, Techno...",[020301],1,[0203]
88,b4b0de57-795f-464c-8fb7-77ca0e391ae4,Rehabilitation Sciences & Physiotherapy,[],1,[]


In [33]:
publications

,ID,Name,Disciplines,WoS ID,Disciplines WoS,vabb ID,Disciplines vabb,Persons,Size,Alias,when_added,Level-2 disciplines
0,3d3661db-6fa2-49d5-9b62-59b0a7829b65,Wind-Structure Interaction Simulations of Oval...,[],[],[],[],[],"[5b652de3-41b8-4a25-81a1-e036dfb8530f, 0ba45b5...",4.0,[],0,[]
1,b3f0df09-9575-4e4c-a938-aa3fd9afdc27,Security and development in EU external relati...,[0506],[],[],[c:vabb:354510],[0506],"[87b376b3-73c2-4fdb-8d74-662d96a506e3, f19619e...",5.0,[],1,[0506]
2,63be7b38-79cb-4352-acae-a0c403928a75,Mechanical strain determines the site-specific...,[],"[000449269700003, 000449269700003]",[],[],[],"[9023ad63-f52c-4180-a946-511ce265c572, 375c304...",19.0,[6a2ebfd0-b8d0-4048-b089-ed7bb93467f1],1,[]
3,0db6ceae-fd5a-4962-ba67-35868707e8ca,Analysis of the drought recovery of Andosols o...,"[0207, 0299, 0207, 0299]","[000379419500018, 000379419500018]","[0207, 0299, 0207, 0299]",[],[],"[85e5549a-2e67-4b62-9833-c5c93116fb36, b73e0e4...",5.5,[e7cc377d-480e-437a-901a-7dd7239666d9],1,"[0207, 0299, 0207, 0299]"
4,b1556823-c194-4cac-86f9-c915bac4c73c,A new unconditionally stable and consistent qu...,"[0207, 0107, 0207, 0107]","[000405997000010, 000405997000010]","[0207, 0107, 0207, 0107]",[],[],"[6d44687b-ee9a-41da-8a2d-919609bb0d4d, 71b6839...",5.0,[0bd28ddb-c4d3-4ad7-8dac-b143c2c65878],1,"[0207, 0107, 0207, 0107]"
...,...,...,...,...,...,...,...,...,...,...,...,...
7445,7ff06fcd-e6fc-4dd5-99ac-fbcecb0f0caa,Thermal and electrical performance of an alkal...,"[0104, 0204, 0299, 0103, 0104, 0204, 0299, 0103]","[000306380200025, 000306380200025]","[0104, 0204, 0299, 0103, 0104, 0204, 0299, 0103]",[],[],"[ea8682b6-8527-41a4-8099-5a077c79cdcc, 783d35e...",7.0,[06f6c529-c469-409e-9ae5-98d85b78bcd3],1,"[0104, 0204, 0299, 0103, 0104, 0204, 0299, 0103]"
7446,4c2a54a2-6b0c-482d-8634-8e64a8ce0606,Long-term recovery of the functional community...,"[0106, 0107]",[000467750400008],"[0106, 0107]",[],[],"[29804705-90a5-4e2b-aeb3-5c1eadbe4170, 00dac63...",11.5,[6045c640-45c2-418f-a09b-b6e94930f44d],1,"[0106, 0107]"
7447,168740f3-8df4-4827-a9dc-5e7aa8a71f71,Performance of 16s rDNA Primer Pairs in the St...,"[0306, 0306]","[000375998700001, 000375998700001]","[0306, 0306]",[],[],"[6c4f337a-6fac-41fc-9224-dc0ccbdeeebb, d5d018a...",7.0,[038a2a15-946c-450d-bb70-29596946b95e],1,"[0306, 0306]"
7448,fe368faf-ff75-435b-9440-82645719e98c,"Effects of conservation agriculture on runoff,...","[0401, 0207, 0401, 0401]","[000294264300014, 000294264300014]","[0401, 0401]",[c:vabb:323976],"[0207, 0401]","[9eb13d63-eba7-4d4e-ad25-8b1ad17a4173, 2a083f7...",11.0,[972c54bf-3ddf-48b6-b702-d3c2864fcedf],1,"[0401, 0207, 0401, 0401]"


# Discipline representation

In [34]:
def make_2D_project_array(project_ID, dm):
    person_IDS = projects_0[projects_0['ID']==project_ID]['Persons'].iloc[0]
    project_date = projects_0[projects_0['ID']==project_ID]['Start'].iloc[0]
    unique_discipline_codes =  unique(projects_0[projects_0['ID']==project_ID]['Level-2 disciplines'].iloc[0])
    if len(unique_discipline_codes)==0:
        return [[project_ID, '/', '/', '/', 'Project has no disciplines']]
    if len(unique_discipline_codes)==1:
        A = unique_discipline_codes[0]
        if expertise(A, person_IDS, project_date):
            expertise_range = A
            explanation = 'Monodisciplinary in discipline' + A + '. There is at least one expert in' + A +'.'
        else:
            expertise_range = '/'
            explanation = 'Monodisciplinary in discipline' + A + '. There is no expert in' + A +'.'
        return [[project_ID, A, '/', expertise_range, 'Monodisciplinary']]
    array=[]
    for (A,B) in list(combinations(unique_discipline_codes, 2)):
        discipline_distance = get_discipline_distance(A, B, dm)
        expertise_range = ''
        if expertise(A, person_IDS, project_date):
            expertise_range= expertise_range + A + '+'
        if expertise(A, person_IDS, project_date):
            expertise_range= expertise_range + B 
        explanation = explain(A, B, expertise(A, person_IDS, project_date), expertise(A, person_IDS, project_date), discipline_distance) 
        array =  array +[[project_ID, A + '+' + B, discipline_distance, expertise_range, explanation]]
    
    
    return array

In [35]:
def explain(A, B, expertiseA, expertiseB, distance):
    if expertiseA and expertiseB:
        if distance>=0.5:
            explanation= 'Possible IDR.'
        elif distance < 0.5:
            explanation = 'Unlikely IDR.'
    else:
        explanation = 'IDR not possible. Expertise in two disciplines is required.'
    return explanation

In [36]:
def expertise(code, person_IDS, project_date):
    for person_ID in person_IDS:
        if is_expert(person_ID, code, project_date):
            return True
    return False

def is_expert(person_ID, code, project_date):
    if project_crit(person_ID, code, project_date):
        return True
    elif organisation_crit(person_ID, code, project_date) and publication_crit(person_ID, code, project_date):
        return True
    else:
        return False
    
def project_crit(person_ID, code, project_date):
    expertise = 0
    project_IDS = persons_0[persons_0['ID']==person_ID]['Projects'].iloc[0]
    for project_ID in project_IDS:
        start_date = projects[projects['ID']==project_ID]['Start'].iloc[0]
        codes = projects[projects['ID']==project_ID]['Level-2 disciplines'].iloc[0]
        duration= projects[projects['ID']==project_ID]['Duration'].iloc[0]
        
        if start_date > project_date:
            continue
        proportion = codes.count(code)/max(1,len(codes))
        if proportion >= 0.5:
            proportion = 1
        expertise=expertise+ duration*proportion
    if expertise >= 48:
        return True

    
def organisation_crit(person_ID, code, project_date):
    expertise = 0
    organisation_IDS = persons_0[persons_0['ID']==person_ID]['Organisations'].iloc[0] 
    for organisation_ID in organisation_IDS:
        begin_date =  dt.date(2011, 9, 5)
        duration=  max(0,difference_in_months(datetime_end = project_date, datetime_start=begin_date))
        codes=organisations[organisations['ID']==organisation_ID]['Level-2 disciplines'].iloc[0]
        
        proportion = codes.count(code)/max(1,len(codes))
        if proportion >= 0.3:
            proportion = 1
        expertise=expertise+ duration*proportion
    if expertise >= 48:
        return True
        
def publication_crit(person_ID, code, project_date):
    expertise = 0
    publication_IDs = persons_0[persons_0['ID']==person_ID]['Publications'].iloc[0] 
    for publication_ID in publication_IDs:
        start_date = dt.date(2011, 9, 5)
        if start_date > project_date:
            continue
        if len(publications[publications['ID']==publication_ID]['Level-2 disciplines']) == 0:
            continue
        codes = publications[publications['ID']==publication_ID]['Level-2 disciplines'].iloc[0]
        
        proportion = codes.count(code)/max(1,len(codes))
        if proportion >= 0.5:
            proportion = 1
        expertise= expertise+proportion
    if expertise >= 1:
        return True

    
def get_discipline_distance(A, B, dm):
    i=discipline_codes.index(A)
    j=discipline_codes.index(B)
    return dm[i,j]

In [37]:
ar=[]
for i in range(len(projects_0)):
    print(projects_0.iloc[i,0])
    ar=ar+make_2D_project_array(projects_0.iloc[i,0], M)
df1 = pd.DataFrame(ar, columns= ['Projects', 'Discipline interaction', 'Discipline distance', 'Expertise', 'Explanation'])
df2 = df1.iloc[:,2:]
df2.index= pd.MultiIndex.from_frame(df1.iloc[:,:2])
df2

fdde0090-f6ed-4938-91a6-13d370125cde
0f80da13-5ac3-45c0-a0df-c2bd93051dbe
0e185133-3ddf-4f8a-81d6-2ddc76a49cb6
fc7f8873-326f-41fe-9c75-08ea05d8058b
9be600ee-e306-4fac-8c4b-0e5134b9ff28
ee843e8f-9c18-4c14-bfee-641e18fbd8e0
bf2e0a13-e0bf-48cd-8339-c8c256770b1c
7a8539e2-93ad-4864-a4c7-405ec848fdd4
70295923-ab2c-44c6-ac9f-2ff4f28b5a75
c04fd85a-d90f-438d-b06a-e5606588ba84
706b3bb1-5ae7-480e-8964-b7f2f051b15e
aee77454-c620-4028-9b3a-fa6983d58965
d57a0eb9-942c-4f13-9570-a966541176e9
e97fc9d0-fc54-432f-95b5-4989bfe11ff7
fbf260f0-ca39-43c0-a66e-471d27da5d5c
f68d198d-f02f-4cf4-8939-c0f5d41bf39f
6532e426-52cb-4da7-bab2-8c3fe53d6aa3
63e6e09e-3425-478e-88f7-7ed2704beffb
1f27dcd3-61fd-4a8f-8a58-afe568ebf9c2
c388b631-fc1f-4a80-8d0c-ffe8269e39ba
1e1fa7e1-16a4-46af-b2f7-d1f998561057
39099cca-b824-4346-81bd-0cff84e379a5
5a480e0c-b39d-455b-9bda-318f348113ab
14e8489c-98c5-4e48-9959-c9ca06ff4343
8a49c146-c7f3-4c49-9919-ad9361b5c425
d72e5b0c-82e5-48d0-b8d1-fc7a8e8fb3cf
09ac1a85-cee7-4c8f-adba-e6d17ba6ea6e
f

Discipline distance  \
Projects                             Discipline interaction                       
fdde0090-f6ed-4938-91a6-13d370125cde 0401+0107                         0.697332   
                                     0401+0105                          0.78369   
                                     0107+0105                         0.357789   
0f80da13-5ac3-45c0-a0df-c2bd93051dbe 0106+0301                         0.572421   
                                     0106+0306                          0.57329   
...                                                                         ...   
f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0 0301+0304                         0.424468   
                                     0302+0304                         0.445105   
1e306f8f-5605-46e1-83f1-19af7819300e 0207                                     /   
f5d40381-b68d-40d1-82e6-57c0409f3caf 0401                                     /   
e5b7dd42-3c2e-485c-ac86-65ec3e2c6080 0502                                     /   

                                                             Expertise  \
Projects                             Discipline interaction              
fdde0090-f6ed-4938-91a6-13d370125cde 0401+0107                           
                                     0401+0105                           
                                     0107+0105                           
0f80da13-5ac3-45c0-a0df-c2bd93051dbe 0106+0301               0106+0301   
                                     0106+0306               0106+0306   
...                                                                ...   
f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0 0301+0304               0301+0304   
                                     0302+0304                           
1e306f8f-5605-46e1-83f1-19af7819300e 0207                         0207   
f5d40381-b68d-40d1-82e6-57c0409f3caf 0401                         0401   
e5b7dd42-3c2e-485c-ac86-65ec3e2c6080 0502                         0502   

                                                                                                   Explanation  
Projects                             Discipline interaction                                                     
fdde0090-f6ed-4938-91a6-13d370125cde 0401+0107               IDR not possible. Expertise in two disciplines...  
                                     0401+0105               IDR not possible. Expertise in two disciplines...  
                                     0107+0105               IDR not possible. Expertise in two disciplines...  
0f80da13-5ac3-45c0-a0df-c2bd93051dbe 0106+0301                                                   Possible IDR.  
                                     0106+0306                                                   Possible IDR.  
...                                                                                                        ...  
f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0 0301+0304                                                   Unlikely IDR.  
                                     0302+0304               IDR not possible. Expertise in two disciplines...  
1e306f8f-5605-46e1-83f1-19af7819300e 0207                                                     Monodisciplinary  
f5d40381-b68d-40d1-82e6-57c0409f3caf 0401                                                     Monodisciplinary  
e5b7dd42-3c2e-485c-ac86-65ec3e2c6080 0502                                                     Monodisciplinary  

[66 rows x 3 columns]

In [38]:
publications[publications['ID']=='7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4']['Level-2 disciplines']

91    [0506, 0507, 0506, 0504]
Name: Level-2 disciplines, dtype: object

In [39]:
person_ID='f19619e2-340d-47ef-9792-35d2436febc4'
is_expert(person_ID,'0506', dt.date(2021, 9, 5)), persons_0[persons_0['ID']==person_ID]['Level-2 disciplines'].iloc[0]

(True, ['0506', '0506', '0506'])

In [40]:
persons_0

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines,Coauthors Publications total count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",6.0
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030...",816.5
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[],244.0
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]",376.0
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060...",117.0
...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,Wout Boerjan,[01061004],"[6c31ab62-2681-4e05-953a-2e439d440a8b, 1e306f8...",[37e63935-be43-4a88-8f83-97da12647a08],"[086fba17-4d80-4006-9f9d-52ed4d090e38, 860b4f9...",[],0,[0106],1144.0
60,4b74615b-d701-4448-a191-cb19436ee4cb,Dirk Vandepitte,[029999],"[017c8b7d-4efe-4647-97af-297bfd23595f, 4a7fc94...","[5f3f66cc-4003-4831-9855-1a9db80e799b, 22358ad...","[f9e8018c-49b3-4d66-9c12-4de3243c097b, e702163...",[],0,[0299],1186.0
61,b6fd60a4-c095-43ed-9743-67cd091db492,Lotte JANSSENS,"[03020301, 03022601, 03022602, 03040201, 03040...","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[172ee4c2-42bc-4e59-93f2-91f28a162f2a, eb6b8bd...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[64b0cfc6-e0a3-4efd-8576-480f849dec0d],0,"[0302, 0302, 0302, 0304, 0304, 0304, 0304, 030...",359.5
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,Nina Goossens,"[03040599, 030126, 030402, 030405, 030626]","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[07fce545-f735-4a6b-9b40-02814a257bca, b4b0de5...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[89a6c8c9-fe61-434e-b2be-402f07e8c751],0,"[0304, 0301, 0304, 0304, 0306]",157.5
